In [ ]:
## import pickle
import numpy as np
import matplotlib.pyplot as plt
from data.visualise import grid_psfs, show_psf_axial
from tifffile import imread
import pandas as pd


def load_pickle_file(dpath):
    with open(dpath, 'rb') as f:
        return pickle.load(f)

    
psfs = imread('/home/miguel/Projects/uni/data/smlm_3d/20230601_MQ_celltype/beads_box15/combined/stacks.ome.tif')
locs = pd.read_hdf('/home/miguel/Projects/uni/data/smlm_3d/20230601_MQ_celltype/beads_box15/combined/locs.hdf', key='locs')


psfs = psfs[:, :, :, :, np.newaxis]

# for i, psf in enumerate(psfs.sum(axis=-1)):
#     plt.title(str(i))
#     show_psf_axial(psf)

exclude_idx = [5, 7, 11, 14, 22, 24, 26, 27, 28, 31, 32, 35, 37, 38, 40, 45, 50, 51, 54, 68, 69, 71, 72, 82, 87, 89, 91, 98, 102, 108, 109, 112, 113, 115, 116, 121, 122, 123, 127, 129, 131, 132, 133, 138, 141, 144, 150, 151, 154, 161, 167, 169, 170, 172, 178, 179, 181, 182, 184, 185, 186, 187, 190, 200, 201, 205, 206, 210, 214, 219, 221, 224, 226, 230, 233, 234, 235, 236, 237, 243]

# exclude_idx = [0, 5, 7, 12, 22, 26, 32, 35, 38, 40, 45, 50, 51, 54, 68, 69, 71, 72, 82, 87, 89, 91, 98, 102, 108, 109, 112, 113, 115, 116, 121, 122, 123, 124, 127, 129, 131, 132, 133, 138, 141, 144, 150, 151, 154, 161, 167, 169, 170, 172, 178, 179, 181, 182, 184, 185, 186, 187, 190, 200, 201, 205, 206, 210, 214, 219, 221, 224, 226, 230, 233, 234, 235, 236, 237, 243]

# print('Excluded PSFs \n\n\n\n\n')
# for i in exclude_idx:
#     show_psf_axial(psfs[i].mean(axis=-1), str(i))
#     plt.plot(psfs[i].max(axis=(1,2)))
#     plt.show()
# print('End of excluded PSFs \n\n\n\n\n')

# for i in range(psfs.shape[0]):
#     if i in exclude_idx:
#         continue
#     plt.title(str(i))
#     show_psf_axial(psfs[i].mean(axis=-1))
#     plt.plot(psfs[i].max(axis=(1,2,3)), label='max')
#     plt.legend()
#     plt.title(str(i))
#     plt.show()
Z_STEP = 20
idx = [i for i in range(psfs.shape[0]) if i not in exclude_idx]

orig_idx = np.arange(psfs.shape[0])
orig_idx = orig_idx[idx]
psfs = psfs[idx]
locs = locs.iloc[idx]
ys = []
for i in range(psfs.shape[0]):
    y = np.arange(psfs.shape[1]) * Z_STEP
    y = y - 1000
    ys.append(y)
ys = np.stack(ys)


In [ ]:
# from data.estimate_offset import estimate_offset

# for i, psf in enumerate(psfs):
#     ys[i] = estimate_offset(psf.squeeze(), (Z_STEP,), disable_boundary_check=True)


In [ ]:
# ROTATION MATCHING ALIGNMENT

# from skimage.filters import gaussian
# from data.align_psfs import norm_zero_one
# from sklearn.metrics import euclidean_distances, mean_squared_error
# import scipy.ndimage as ndi


# Z_STEP = 20
# UPSCALE_RATIO = 5
        
# def find_seed_psf(df):
#     n_points = 5
#     # Seed PSF - most centered PSF in FOV
#     center = df[['x', 'y']].mean(axis=0).to_numpy()
#     coords = df[['x', 'y']].to_numpy()
#     dists = euclidean_distances([center], coords).squeeze()
#     return np.argsort(dists)[:n_points]
# #     first_point = np.argmin(dists)
# #     return first_point


# def norm_sum_imgs(psf):
#     psf_sums = psf.sum(axis=(1,2, 3))
#     psf = psf / psf_sums[:, np.newaxis, np.newaxis, np.newaxis]
#     return psf


# def eval_roll(fixed, moving, roll, debug=False):
#     fixed_section = fixed
#     moving_section = moving    
#     if roll < 0:
#         moving_section = moving[-roll:]
#         fixed_section = fixed_section[:moving_section.shape[0]]
#     elif roll > 0:
#         fixed_section = fixed[roll:]
#         moving_section = moving_section[:fixed_section.shape[0]]

#     score = mean_squared_error(fixed_section.flatten(), moving_section.flatten())
# #     score = ((fixed_section-moving_section)**2).sum()
#     if debug:
#         plt.rcParams['figure.figsize'] = [10, 3]
#         x = np.arange(fixed.shape[0])
#         x_moving = np.arange(moving.shape[0]) + roll
#         plt.title(f'full roll {roll}')
        
#         plt.plot(x, moving.max(axis=(1,2)), label='original')
#         plt.plot(x_moving, moving.max(axis=(1,2)), label='shifted')
#         plt.plot(x, fixed.max(axis=(1,2)), label='seed')
#         plt.xlim((-5, max((x.max(), x_moving.max()))))
#         plt.legend()
#         plt.show()
        
# #         plt.title(f'{round(score, 2)} length {moving_section.shape[0]} roll {roll}')
# #         plt.plot(moving_section.max(axis=(1,2)), label='moving')
# #         plt.plot(fixed_section.max(axis=(1,2)), label='fixed')
# #         plt.legend()
# #         plt.show()   
#         show_psf_axial(fixed.mean(axis=-1), 'fixed', 3*UPSCALE_RATIO)
#         show_psf_axial(moving.mean(axis=-1), 'moving', 3*UPSCALE_RATIO)
#         show_psf_axial(np.roll(moving.mean(axis=-1), roll, 0), f'shifted {roll}', 3*UPSCALE_RATIO)
        

#     return score

# def find_best_roll(fixed, moving):
#     roll_min = -fixed.shape[0] // 2
#     roll_max = fixed.shape[0] // 2
#     best_roll = None
#     min_score = np.inf
    
#     rolls = []
#     scores = []
#     for roll in range(roll_min, roll_max, 1):
#         score = eval_roll(fixed, moving, roll, debug=False)
#         if score < min_score:
#             best_roll = roll
#             min_score = score
#         rolls.append(roll)
#         scores.append(score)
        
# #     plt.plot(rolls, scores)
# #     plt.xlabel('roll')
# #     plt.ylabel('MSE')
# #     plt.show()
#     eval_roll(fixed, moving, best_roll, debug=True)
#     return best_roll * (Z_STEP/UPSCALE_RATIO)


# def upscale_psf(psf):
#     out = ndi.zoom(psf, (UPSCALE_RATIO, 1, 1, 1), order=2)
#     return out
    
# def register_psfs(psfs, df):
#     GAUSS_SIGMA = 1
#     offsets = []
#     seed_idx = find_seed_psf(df)
# #     seed_idx = -1
# #     seed_psf = norm_zero_one(psfs.mean(axis=0))
#     seed_psf = norm_zero_one(gaussian(psfs[seed_idx].mean(axis=0), sigma=GAUSS_SIGMA))
#     seed_psf = upscale_psf(seed_psf)
#     seed_idx = -1
    
#     for i in range(psfs.shape[0]):
#         if i == seed_idx:
#             offset = 0
#         else:
#             psf = norm_zero_one(gaussian(psfs[i], sigma=GAUSS_SIGMA))
#             psf = upscale_psf(psf)
# #             plt.plot(psf.max(axis=(1,2,3)))
# #             plt.plot(seed_psf.max(axis=(1,2,3)))
# #             plt.show()
#             offset = find_best_roll(seed_psf, psf)
#         offsets.append(offset)
#     return np.array(offsets)

# offsets = register_psfs(psfs, locs)


In [ ]:
# SIMPLE ITK alignment

# import SimpleITK as sitk
# import numpy as np
# from data.align_psfs import norm_zero_one
# import pandas as pd
# from sklearn.metrics import euclidean_distances

# Z_STEP = 20 
# def command_iteration(method):
#     print(
#         f"{method.GetOptimizerIteration():3} "
#         + f" = {method.GetMetricValue():7.5f} "
#         + f" : {method.GetOptimizerPosition()}"
#     )        

# def arr_to_img(arr):
#     arr = norm_zero_one(arr.squeeze()).astype(np.float32)
#     img = sitk.GetImageFromArray(arr)
#     img = sitk.DiscreteGaussian(img)
#     return img


# def register_psf(psf, fixed):
#     moving = arr_to_img(psf)

#     R = sitk.ImageRegistrationMethod()

#     R.SetMetricAsMeanSquares()

#     R.SetOptimizerAsRegularStepGradientDescent(
#         learningRate=0.05,
#         minStep=1e-12,
#         numberOfIterations=10000,
#         gradientMagnitudeTolerance=1e-12,
#     )
#     R.SetOptimizerScalesFromIndexShift()

#     R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))

#     R.SetInterpolator(sitk.sitkLinear)

# #     R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

#     outTx = R.Execute(fixed, moving)
#     print("-------")
#     print(outTx)
#     print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
#     print(f" Iteration: {R.GetOptimizerIteration()}")
#     print(f" Metric value: {R.GetMetricValue()}")

#     return outTx.GetOffset()[2] * Z_STEP


# def find_seed_psf(df):
#     # Seed PSF - most centered PSF in FOV
#     center = df[['x', 'y']].mean(axis=0).to_numpy()
#     coords = df[['x', 'y']].to_numpy()
#     dists = euclidean_distances([center], coords).squeeze()
#     first_point = np.argmin(dists)
#     return first_point

# def register_psfs(psfs, df):
#     offsets = []
#     seed_idx = find_seed_psf(df)
    
#     ref = arr_to_img(psfs[seed_idx])

#     for i in range(psfs.shape[0]):
#         if i == seed_idx:
#             offset = 0
#         else:
#             offset = register_psf(psfs[i], ref)
#         offsets.append(offset)
#     return offsets


# # offsets = register_psfs(psfs, locs)
# # offsets = np.array(offsets)
# # offset_backup = offsets.copy()

In [ ]:
from scipy.interpolate import UnivariateSpline
from data.align_psfs import norm_zero_one

UPSCALE_RATIO = 10
Z_STEP = 20

def find_peak(psf):
    x = np.arange(psf.shape[0]) * Z_STEP
    inten = norm_zero_one(psf.max(axis=(1,2)))
#   prev 0.8
    cs = UnivariateSpline(x, inten, k=3, s=1.25)

    x_ups = np.linspace(0, psf.shape[0], len(x) * UPSCALE_RATIO) * Z_STEP
    


    peak = x_ups[np.argmax(cs(x_ups))] 
    
    plt.plot(x-peak, inten, label='raw')
    plt.plot(x_ups-peak, cs(x_ups), label='fit')
    plt.legend()
    plt.show()
    return peak

offsets = np.array([find_peak(psf) for psf in psfs])

In [ ]:
# offsets = [0] * psfs.shape[0]
for i in range(len(offsets)):
    ys[i] = ys[i] - offsets[i]

In [ ]:
# ys = []
# for i, offset in zip(list(range(psfs.shape[0])), offsets):
#     y = np.arange(psfs.shape[1]) * Z_STEP
#     ys.append(y-offset)
# ys = np.stack(ys)


In [ ]:
# valid_idx = np.argwhere(abs(offsets)<750).squeeze()
# psfs = psfs[valid_idx]
# coords = coords[valid_idx]
# ys = ys[valid_idx]

# offsets = offsets[valid_idx]

In [ ]:
# Re-center z positions using average peak

peak_zs = []
for psf, z in zip(psfs, ys):
    peak = np.argmax(psf.max(axis=(1,2,3)))
    peak_zs.append(z[peak])
    
mean_peak = np.mean(peak_zs)
print(mean_peak)
ys = ys - mean_peak



In [ ]:
from data.align_psfs import norm_zero_one
from skimage.filters import gaussian

plt.rcParams['figure.figsize'] = [10, 3]
for psf, z in list(zip(psfs, ys))[0:10]:
    y = norm_zero_one(gaussian(psf).max(axis=(1,2,3))).squeeze()
#     show_psf_axial(psf.squeeze(), '', 1)
    plt.plot(z, y)
plt.xlim((-1000, 1000))
plt.show()

In [ ]:
import numpy as np

def concat_training_datasets(X_train, y_train, X_val, y_val, X_test, y_test):
    X_train = list(X_train)
    X_val = list(X_val)
    X_test = list(X_test)
    for i in range(2):
        X_train[i] = np.concatenate(X_train[i]).astype(float)
        X_val[i] = np.concatenate(X_val[i]).astype(float)
        X_test[i] = np.concatenate(X_test[i]).astype(float)

    y_train = np.concatenate(y_train).astype(float)
    y_val = np.concatenate(y_val).astype(float)
    y_test = np.concatenate(y_test).astype(float)

#     print(X_train[0].shape, X_train[1].shape)
#     print(X_val[0].shape, X_val[1].shape)
#     print(X_test[0].shape, X_test[1].shape)


#     print(y_train.shape)
#     print(y_val.shape)
#     print(y_test.shape)
    return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
# Trim stacks

Z_RANGE = 1000
def filter_z_datasets(X_train, y_train, X_val, y_val, X_test, y_test):
    def filter_z_range(X, zs):
        psfs, coords = X
        print(psfs.shape, coords.shape, zs.shape)
        valid_ids = np.argwhere(abs(zs.squeeze()) < Z_RANGE).squeeze()
        return [psfs[valid_ids], coords[valid_ids]], zs[valid_ids]

    X_train, y_train = filter_z_range(X_train, y_train)
    X_val, y_val = filter_z_range(X_val, y_val)
    X_test, y_test = filter_z_range(X_test, y_test)

#     print(X_train[0].min(), X_train[0].mean(), X_train[0].max())
#     print(X_val[0].min(), X_val[0].mean(), X_val[0].max())
#     print(X_test[0].min(), X_test[0].mean(), X_test[0].max())

#     print(y_train.min(), y_train.max())
#     print(y_val.min(), y_val.max())
#     print(y_test.min(), y_test.max())

#     print(X_train[0].shape, X_train[1].shape)
#     print(X_val[0].shape, X_val[1].shape)
#     print(X_test[0].shape, X_test[1].shape)


#     print(y_train.shape)
#     print(y_val.shape)
#     print(y_test.shape)
    
    return X_train, y_train, X_val, y_val, X_test, y_test
    


In [ ]:
# data augmentation


from tensorflow.keras import layers, Sequential
from data.visualise import grid_psfs

def aug_dataset(X_train, y_train):
    AUG_RATIO = 0.5
    MAX_TRANSLATION_PX = 2
    MAX_GAUSS_NOISE = 0.001
    img_size = X_train[0].shape[1]

    aug_pipeline = Sequential([
        layers.GaussianNoise(stddev=MAX_GAUSS_NOISE*X_train[0].max(), seed=42),
        layers.RandomTranslation(MAX_TRANSLATION_PX/img_size, MAX_TRANSLATION_PX/img_size, seed=42),
        layers.RandomBrightness(0.2, [X_train[0].min(), X_train[0].max()], seed=42)
    ])

    idx = np.random.randint(0, X_train[0].shape[0], size=int(AUG_RATIO*X_train[0].shape[0]))

    aug_psfs = aug_pipeline(X_train[0][idx].copy(), training=True).numpy()
    aug_coords = X_train[1][idx]

    aug_z = y_train[idx]

    subset_psfs = np.concatenate((aug_psfs[0:100], X_train[0][idx][0:100]))
    plt.imshow(grid_psfs(subset_psfs.mean(axis=-1)))
    plt.show()

    train_psfs = np.concatenate([aug_psfs, X_train[0]])
    train_coords = np.concatenate([aug_coords, X_train[1]])
    train_zs = np.concatenate([aug_z, y_train])

    X_train = [train_psfs, train_coords]
    y_train = train_zs
    return X_train, y_train
    


In [ ]:
# from skimage.transform import rescale
# plt.rcParams['figure.figsize'] = [3, 3]
# def bin_pixels(img):
#     subsample_ratio = 2
#     new_shape = [s//subsample_ratio for s in img.shape[0:2]] + [1]
#     new_img = rescale(img, 1/subsample_ratio, anti_aliasing=False)
    
#     upscaled_img = rescale(new_img, (subsample_ratio, subsample_ratio, 1), anti_aliasing=False, order=0)
# #     plt.imshow(img)
# #     plt.show()
# #     plt.imshow(new_img)
# #     plt.title(new_img.shape)
# #     plt.show()
    
# #     plt.imshow(upscaled_img)
# #     plt.title(upscaled_img.shape)
# #     plt.show()
#     return upscaled_img

# plt.imshow(X_train[0])
# plt.show()
# X_train = np.stack([bin_pixels(img) for img in X_train])
# X_val = np.stack([bin_pixels(img) for img in X_val])
# X_test = np.stack([bin_pixels(img) for img in X_test])
# plt.imshow(X_train[0])
# plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def min_max_xy_coords(X_train, X_val, X_test):
    coords_scaler = MinMaxScaler().fit(X_train[1])
    X_train[1] = coords_scaler.transform(X_train[1])
    X_val[1] = coords_scaler.transform(X_val[1])
    X_test[1] = coords_scaler.transform(X_test[1])
    return coords_scaler





In [ ]:
from skimage.transform import resize

def resize_psfs(X_train, X_val, X_test):
    target_size = 128
    imshape = (target_size, target_size, 3)

    X_train[0] = np.stack([resize(psf, imshape) for psf in X_train[0]])
    X_val[0] = np.stack([resize(psf, imshape) for psf in X_val[0]])
    X_test[0] = np.stack([resize(psf, imshape) for psf in X_test[0]])
    


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_imgs(X_train, X_val, X_test):
    datagen = ImageDataGenerator(
        rescale=1.0/65336.0,
        samplewise_center=False,
        samplewise_std_normalization=False,
        featurewise_center=True,
        featurewise_std_normalization=True,
        horizontal_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train[0])

    X_train_preproc = [X_train[0].copy(), X_train[1].copy()]
    X_val_preproc = [X_val[0].copy(), X_val[1].copy()]
    X_test_preproc = [X_test[0].copy(), X_test[1].copy()]

    X_train_preproc[0] = datagen.standardize(X_train_preproc[0])
    X_val_preproc[0] = datagen.standardize(X_val_preproc[0])
    X_test_preproc[0] = datagen.standardize(X_test_preproc[0])
    
    return X_train_preproc, X_val_preproc, X_test_preproc, datagen


# X_train_preproc, X_val_preproc, X_test_preproc, datagen = preprocess_imgs(X_train, X_val, X_test)

# preprocessors = {
#     'psfs': datagen,
#     'coords': coords_scaler
# }

# import pickle
# with open('./scalers.p', 'wb') as f:
#     pickle.dump(preprocessors, f)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tqdm.keras import TqdmCallback
from tensorflow.keras import regularizers

import tensorflow as tf

def train_model(X_train_preproc, y_train, X_val_preproc, y_val):
    img_input = layers.Input((X_train_preproc[0][0].shape))
    coords_input = layers.Input(X_train_preproc[1][0].shape)

    x = img_input

    x = keras.applications.MobileNet(
        input_tensor=img_input,
        include_top=False,
        weights='./mobilenet_1_0_128_tf_no_top.h5',
        pooling='avg',
    )(x)

    x = tf.concat([x, coords_input], axis=-1)

    x = layers.Dropout(0.5)(x)
    out = layers.Dense(1, activation="linear",
                       kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                       bias_regularizer=regularizers.L2(1e-4),
                       activity_regularizer=regularizers.L2(1e-5)
                      )(x)

    model = keras.Model(inputs=(img_input, coords_input), outputs=out)

    model.summary(expand_nested=False)


    batch_size = 32
    epochs = 5000
    lr = 0.001

    model.compile(loss='mean_squared_error', optimizer=optimizers.AdamW(learning_rate=lr), metrics=['mean_absolute_error'])

    callbacks = [
        ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.1,
                          patience=25, verbose=True, mode='min', min_delta=5, min_lr=1e-6,),
        EarlyStopping(monitor='val_mean_absolute_error', patience=75,
                      verbose=False, min_delta=1, restore_best_weights=True),
        TqdmCallback(verbose=1),
    ]


    history = model.fit(X_train_preproc, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_preproc, y_val), callbacks=callbacks, shuffle=True, verbose=False)

    model.save('./latest_model')
    return model, history


# Peak alignment
# 5.424, 114, 95
# w/ aug 5.19, 113, 96
# w/ more data and brightness aug 5.5, 119.52, 89
# Using smoothed peaks (s=1.25) 63, 74, 67
# 9.396, 78, 64.52


In [ ]:
# from sklearn.metrics import mean_absolute_error

def measure_errors(X_train, y_train, X_val, y_val):
    for k, (X, y) in [('train', (X_train, y_train)), ('val', (X_val, y_val))]:
        res = model.predict(X, verbose=True)
        error = mean_absolute_error(res, y)
        print(k, round(error, 3))

In [ ]:
import matplotlib.pyplot as plt

def plt_history(history):
    plt.rcParams['figure.figsize'] = [5, 5]
    fig, ax1 = plt.subplots()
    ax1.plot(history.history['mean_absolute_error'], label='mse')
    ax1.plot(history.history['val_mean_absolute_error'], label='val_mse')
    ax1.set_ylim([0, 500])
    ax1.legend(loc=1)
    ax2 = ax1.twinx()
    ax2.plot(history.history['lr'], label='lr', color='red')
    ax2.legend(loc=0)


In [ ]:
from sklearn.metrics import mean_absolute_error
import scipy.optimize as opt

# plt.rcParams['figure.figsize'] = [5, 3]

def chunk_dataset(train, z):
    splits = [0] + list(np.argwhere(np.diff(z.squeeze()) < 0).flatten()+1) + [z.shape[0]]
    for i in range(len(splits)-1):
        start = splits[i]
        end = splits[i+1]
        yield (train[0][start:end], train[1][start:end]), z[start:end]
        
def bestfit_error(z_true, z_pred):

    def linfit(x, c):
        return x + c

    x = z_true.squeeze()
    y = z_pred.squeeze()
    popt, _ = opt.curve_fit(linfit, x, y, p0=[0])

    x = np.linspace(z_true.min(), z_true.max(), len(y))
    y_fit = linfit(x, popt[0])
#     plt.plot(x, x, label=f'x=y')
#     plt.plot(x, y_fit, label=f'best_fit c={popt[0]}')
#     plt.show()
    error = mean_absolute_error(y_fit, y)
    return error, popt[0]

def get_offsets(X, y):
    offsets = []
    for i, (inp, true_zs) in enumerate(chunk_dataset(X, y)):
        pred_zs = model.predict(inp, verbose=False)
        error, offset = bestfit_error(true_zs, pred_zs)
        offsets.append(offset)
    return offsets
        
    

# res_offsets = []
# for name, dataset in [
# #     ('train', (X_train_preproc, y_train)), 
#     ('val', (X_val_preproc, y_val)), 
#     ('test', (X_test_preproc, y_test))]:
#     inputs, z = dataset
#     for i, (inp, true_zs) in enumerate(chunk_dataset(*dataset)):
#         pred_zs = model.predict(inp)
#         show_psf_axial(inp[0].mean(axis=-1))
#         find_peak(inp[0].mean(axis=-1))
        
#         error, offset = bestfit_error(true_zs, pred_zs)
#         res_offsets.append(abs(offset))
# #         error = mean_absolute_error(true_zs, pred_zs)
#         plt.title(f'variable error {round(error, 1)}nm, const error {round(offset, 1)}nm')
#         plt.scatter(true_zs, pred_zs, label=f'{name}_{i}', marker='+', c='orange')
#         plt.legend()
#         plt.show()
#         if name == 'val':
#             idx = val_idx
#         else:
#             idx = test_idx
#         psf_idx = idx[i]
#         psf = psfs[psf_idx]
#         plt.plot(psf.max(axis=(1,2,3)))
#         plt.title(str(psf_idx))
#         plt.show()

# print('Mean offset', round(np.mean(res_offsets), 3))
    


In [ ]:
import gc

coords = locs[['x', 'y']].to_numpy()[:, np.newaxis]
coords = np.repeat(coords, psfs.shape[1], 1)

print(psfs.shape, ys.shape, coords.shape)
from sklearn.model_selection import KFold

n_folds = 5
kfold = KFold(n_folds, shuffle=True, random_state=42)
idx = np.arange(psfs.shape[0])

results = np.zeros((n_folds, len(idx)), dtype=object)
results[:] = np.nan
for split, (train_idx, val_idx) in enumerate(kfold.split(idx)):
    gc.collect()
    X_train = psfs[train_idx], coords[train_idx]
    y_train = ys[train_idx]

    X_val = psfs[val_idx], coords[val_idx]
    y_val = ys[val_idx]
    
    X_test = psfs[val_idx].copy(), coords[val_idx].copy()
    y_test = ys[val_idx].copy()
    X_train, y_train, X_val, y_val, X_test, y_test = concat_training_datasets(X_train, y_train, X_val, y_val, X_test, y_test)

    X_train, y_train, X_val, y_val, X_test, y_test = filter_z_datasets(X_train, y_train, X_val, y_val, X_test, y_test)
    X_train, y_train = aug_dataset(X_train, y_train)
    coords_scaler = min_max_xy_coords(X_train, X_val, X_test)
    resize_psfs(X_train, X_val, X_test)
    X_train_preproc, X_val_preproc, X_test_preproc, datagen = preprocess_imgs(X_train, X_val, X_test)
    
    model, history = train_model(X_train_preproc, y_train, X_val_preproc, y_val)
    plt_history(history)
    
    orig_train_idx = orig_idx[train_idx]
    orig_val_idx = orig_idx[val_idx]
    
    results[split, train_idx] = 'train'

    offsets = get_offsets(X_val, y_val)
    results[split, val_idx] = offsets

    
    

In [ ]:
res = results.T
for i, row in enumerate(res):
    val = [r for r in row if isinstance(r, np.float64)][0]
    print(val)
    if abs(val) < 25:
        plt.plot(psfs[i].max(axis=(1,2,3)))
        plt.show()

In [ ]:
# import pandas as pd

# df = pd.read_csv('/home/miguel/Projects/uni/data/smlm_3d/20230601_MQ_celltype/nup/fov2/storm_1/storm_1_MMStack_Default.ome_locs_40k.csv')

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.scatterplot(data=df, x='x [nm]', y='y [nm]', alpha=0.05)

# xlims = [31000, 33000]
# ylims = [24000, 26000]
# plt.xlim(*xlims)
# plt.ylim(*ylims)
# plt.show()

# pixel_size = 86
# xlim_pixels = [x/pixel_size for x in xlims]
# ylim_pixels= [y/pixel_size for y in ylims]

# print(xlim_pixels)
# print(ylim_pixels)
